<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **SpaceX  Falcon 9 first stage Landing Prediction**


# Lab 1: Collecting the data


Estimated time needed: **45** minutes


In this capstone, we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/lab_v2/images/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


In this lab, you will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Import Libraries and Define Auxiliary Functions


We will import the following libraries into the lab


In [35]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [57]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(x)).json()
            BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [58]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/" + str(x)).json()
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])
            LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [59]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load_list in data['payloads']:
        if load_list:
            load = load_list[0]  # ← Pega o primeiro da lista
            response = requests.get("https://api.spacexdata.com/v4/payloads/" + load).json()
            PayloadMass.append(response['mass_kg'])
            Orbit.append(response['orbit'])
        else:
            PayloadMass.append(None)
            Orbit.append(None)

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [60]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core_list in data['cores']:
        if core_list and len(core_list) > 0:
            core = core_list[0]  # ← Pega o primeiro da lista
            
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/" + core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            
            Outcome.append(str(core['landing_success']) + ' ' + str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)
            Outcome.append(None)
            Flights.append(None)
            GridFins.append(None)
            Reused.append(None)
            Legs.append(None)
            LandingPad.append(None)


Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [61]:
spacex_url = "https://api.spacexdata.com/v4/launches/past"
response = requests.get(spacex_url)
print(f"Status: {response.status_code}")

data = response.json()
df = pd.DataFrame(data)
print(f"Lançamentos: {len(df)}")


Status: 200
Lançamentos: 187


Check the content of the response


In [70]:
print("\n1️⃣ Extraindo versão do booster...")
getBoosterVersion(df)
print(f"   ✅ {len(BoosterVersion)} boosters extraídos")

print("\n2️⃣ Extraindo dados do local de lançamento...")
getLaunchSite(df)
print(f"   ✅ {len(LaunchSite)} locais extraídos")

print("\n3️⃣ Extraindo dados da carga útil...")
getPayloadData(df)
print(f"   ✅ {len(PayloadMass)} payloads extraídos")

print("\n4️⃣ Extraindo dados do core...")
getCoreData(df)
print(f"   ✅ {len(Outcome)} cores extraídos")



1️⃣ Extraindo versão do booster...
   ✅ 187 boosters extraídos

2️⃣ Extraindo dados do local de lançamento...
   ✅ 187 locais extraídos

3️⃣ Extraindo dados da carga útil...
   ✅ 187 payloads extraídos

4️⃣ Extraindo dados do core...
   ✅ 187 cores extraídos


In [73]:
print(f"Tamanho do DataFrame: {len(df)}")
print(f"Colunas do DataFrame: {df.shape}")
print(f"\nPrimeiras 5 linhas:")
print(df[['flight_number', 'rocket', 'date_utc']].head())
print(f"\nÚltimas 5 linhas:")
print(df[['flight_number', 'rocket', 'date_utc']].tail())


Tamanho do DataFrame: 187
Colunas do DataFrame: (187, 27)

Primeiras 5 linhas:
   flight_number                    rocket                  date_utc
0              1  5e9d0d95eda69955f709d1eb  2006-03-24T22:30:00.000Z
1              2  5e9d0d95eda69955f709d1eb  2007-03-21T01:10:00.000Z
2              3  5e9d0d95eda69955f709d1eb  2008-08-03T03:34:00.000Z
3              4  5e9d0d95eda69955f709d1eb  2008-09-28T23:15:00.000Z
4              5  5e9d0d95eda69955f709d1eb  2009-07-13T03:35:00.000Z

Últimas 5 linhas:
     flight_number                    rocket                  date_utc
182            183  5e9d0d95eda69973a809d1ec  2022-09-05T02:09:00.000Z
183            184  5e9d0d95eda69973a809d1ec  2022-09-11T01:10:00.000Z
184            185  5e9d0d95eda69973a809d1ec  2022-09-17T01:05:00.000Z
185            186  5e9d0d95eda69973a809d1ec  2022-09-24T23:30:00.000Z
186            187  5e9d0d95eda69973a809d1ec  2022-10-05T16:00:00.000Z


In [71]:
# Ver os primeiros 5 resultados
print("\nPrimeiros 5 cores:")
for i in range(5):
    print(f"{i}: Block={Block[i]}, Serial={Serial[i]}, Outcome={Outcome[i]}")



Primeiros 5 cores:
0: Block=None, Serial=Merlin1A, Outcome=None None
1: Block=None, Serial=Merlin2A, Outcome=None None
2: Block=None, Serial=Merlin1C, Outcome=None None
3: Block=None, Serial=Merlin2C, Outcome=None None
4: Block=None, Serial=Merlin3C, Outcome=None None


You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


### Task 1: Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [11]:
# Use json_normalize meethod to convert the json result into a dataframe

We should see that the request was successfull with the 200 status response code


In [15]:
# Get the head of the dataframe

Now we decode the response content as a Json using <code>.json()</code> and turn it into a Pandas dataframe using <code>.json_normalize()</code>


In [80]:
# Converter a resposta em JSON
data = response.json()

Using the dataframe <code>data</code> print the first 5 rows


In [81]:
print("\n" + "="*70)
print("ANÁLISES DOS DADOS")
print("="*70)

# 1. Taxa de reutilização
reused_count = df_final['Reused'].sum()
total = len(df_final)
reuse_rate = (reused_count / total) * 100

print(f"\n1️⃣ REUTILIZAÇÃO:")
print(f"   Boosters reutilizados: {reused_count} / {total}")
print(f"   Taxa de reutilização: {reuse_rate:.1f}%")

# 2. Tipos de boosters
print(f"\n2️⃣ TIPOS DE BOOSTERS:")
booster_counts = df_final['Booster Version'].value_counts()
for booster, count in booster_counts.items():
    pct = (count / total) * 100
    print(f"   • {booster}: {count} ({pct:.1f}%)")

# 3. Locais de lançamento
print(f"\n3️⃣ LOCAIS DE LANÇAMENTO (TOP 5):")
launch_sites = df_final['Launch Site'].value_counts().head(5)
for site, count in launch_sites.items():
    print(f"   • {site}: {count}")

# 4. Órbitas mais comuns
print(f"\n4️⃣ ÓRBITAS MAIS COMUNS:")
orbits = df_final['Orbit'].value_counts().head(5)
for orbit, count in orbits.items():
    print(f"   • {orbit}: {count}")

# 5. Massa média dos payloads
payload_mass_numeric = pd.to_numeric(df_final['Payload Mass'], errors='coerce')
print(f"\n5️⃣ PAYLOAD MASS:")
print(f"   Mínimo: {payload_mass_numeric.min():.0f} kg")
print(f"   Máximo: {payload_mass_numeric.max():.0f} kg")
print(f"   Média: {payload_mass_numeric.mean():.0f} kg")

# 6. Locais de pouso
print(f"\n6️⃣ LOCAIS DE POUSO:")
landing_pads = df_final['LandingPad'].value_counts().head(5)
for pad, count in landing_pads.items():
    if pad and str(pad) != 'nan':
        print(f"   • {pad}: {count}")

print("\n" + "="*70)
print("✅ LABORATÓRIO CONCLUÍDO COM SUCESSO!")
print("="*70)



ANÁLISES DOS DADOS

1️⃣ REUTILIZAÇÃO:
   Boosters reutilizados: 115 / 187
   Taxa de reutilização: 61.5%

2️⃣ TIPOS DE BOOSTERS:
   • Falcon 9: 179 (95.7%)
   • Falcon 1: 5 (2.7%)
   • Falcon Heavy: 3 (1.6%)

3️⃣ LOCAIS DE LANÇAMENTO (TOP 5):
   • CCSFS SLC 40: 99
   • KSC LC 39A: 55
   • VAFB SLC 4E: 28
   • Kwajalein Atoll: 5

4️⃣ ÓRBITAS MAIS COMUNS:
   • VLEO: 58
   • GTO: 35
   • ISS: 33
   • LEO: 20
   • PO: 14

5️⃣ PAYLOAD MASS:
   Mínimo: 20 kg
   Máximo: 15600 kg
   Média: 7868 kg

6️⃣ LOCAIS DE POUSO:
   • 5e9e3032383ecb6bb234e7ca: 64
   • 5e9e3033383ecbb9e534e7cc: 40
   • 5e9e3033383ecb075134e7cd: 21
   • 5e9e3032383ecb267a34e7c7: 18
   • 5e9e3032383ecb554034e7c9: 6

✅ LABORATÓRIO CONCLUÍDO COM SUCESSO!


In [84]:
# Salvar o DataFrame final
df_final.to_csv('spacex_api_data.csv', index=False)

print("\n✅ Arquivo 'spacex_api_data.csv' salvo com sucesso!")
print(f"   Total de registros: {len(df_final)}")
print(f"   Total de colunas: {len(df_final.columns)}")



✅ Arquivo 'spacex_api_data.csv' salvo com sucesso!
   Total de registros: 187
   Total de colunas: 17


You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API again to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [85]:
import datetime

print("="*70)
print("LIMPEZA E FILTRAGEM DOS DADOS")
print("="*70)

# PASSO 1: Fazer backup do DataFrame original
df_raw = df.copy()
print(f"\n1️⃣ Backup criado:")
print(f"   Linhas originais: {len(df_raw)}")

# PASSO 2: Manter apenas colunas importantes
data = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']].copy()
print(f"\n2️⃣ Colunas selecionadas (6 colunas)")

# PASSO 3: Remover linhas com múltiplos cores (Falcon Heavy tem 2 cores extras)
print(f"\n3️⃣ Filtrando múltiplos cores:")
print(f"   Antes: {len(data)} linhas")
data = data[data['cores'].map(len) == 1]
print(f"   Depois: {len(data)} linhas")
print(f"   Removidas: {len(df_raw) - len(data)} linhas (Falcon Heavy)")

# PASSO 4: Remover linhas com múltiplos payloads
print(f"\n4️⃣ Filtrando múltiplos payloads:")
print(f"   Antes: {len(data)} linhas")
data = data[data['payloads'].map(len) == 1]
print(f"   Depois: {len(data)} linhas")
print(f"   Removidas: linhas com múltiplos payloads")

# PASSO 5: Extrair valores únicos das listas
print(f"\n5️⃣ Extraindo valores das listas:")
data['cores'] = data['cores'].map(lambda x: x[0])
data['payloads'] = data['payloads'].map(lambda x: x[0])
print(f"   ✅ Cores extraídos (de lista para string)")
print(f"   ✅ Payloads extraídos (de lista para string)")

# PASSO 6: Converter datas
print(f"\n6️⃣ Convertendo datas:")
data['date'] = pd.to_datetime(data['date_utc']).dt.date
print(f"   ✅ Coluna 'date' criada no formato YYYY-MM-DD")
print(f"   Primeira data: {data['date'].min()}")
print(f"   Última data: {data['date'].max()}")

# PASSO 7: Filtrar por data (até 13 de novembro de 2020)
print(f"\n7️⃣ Filtrando por data (até 13/11/2020):")
print(f"   Antes: {len(data)} linhas")
data = data[data['date'] <= datetime.date(2020, 11, 13)]
print(f"   Depois: {len(data)} linhas")
print(f"   Removidas: linhas após 13/11/2020")

print(f"\n" + "="*70)
print(f"✅ LIMPEZA CONCLUÍDA!")
print(f"="*70)
print(f"\n📊 Resumo final:")
print(f"   Linhas no dataset limpo: {len(data)}")
print(f"   Colunas: {list(data.columns)}")
print(f"   Período: {data['date'].min()} a {data['date'].max()}")

# PASSO 8: Visualizar o resultado
print(f"\n📋 Primeiras 5 linhas do dataset limpo:")
print(data.head())

print(f"\n📈 Informações do dataset:")
print(data.info())


LIMPEZA E FILTRAGEM DOS DADOS

1️⃣ Backup criado:
   Linhas originais: 187

2️⃣ Colunas selecionadas (6 colunas)

3️⃣ Filtrando múltiplos cores:
   Antes: 187 linhas
   Depois: 184 linhas
   Removidas: 3 linhas (Falcon Heavy)

4️⃣ Filtrando múltiplos payloads:
   Antes: 184 linhas
   Depois: 172 linhas
   Removidas: linhas com múltiplos payloads

5️⃣ Extraindo valores das listas:
   ✅ Cores extraídos (de lista para string)
   ✅ Payloads extraídos (de lista para string)

6️⃣ Convertendo datas:
   ✅ Coluna 'date' criada no formato YYYY-MM-DD
   Primeira data: 2006-03-24
   Última data: 2022-10-05

7️⃣ Filtrando por data (até 13/11/2020):
   Antes: 172 linhas
   Depois: 94 linhas
   Removidas: linhas após 13/11/2020

✅ LIMPEZA CONCLUÍDA!

📊 Resumo final:
   Linhas no dataset limpo: 94
   Colunas: ['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc', 'date']
   Período: 2006-03-24 a 2020-11-05

📋 Primeiras 5 linhas do dataset limpo:
                     rocket          

In [86]:
# Salvar o dataset limpo
data.to_csv('spacex_data_cleaned.csv', index=False)
print("✅ Dataset limpo salvo como 'spacex_data_cleaned.csv'")

# Ver estatísticas
print(f"\n📊 ESTATÍSTICAS:")
print(f"   Redução de dados: {(1 - len(data)/len(df_raw))*100:.1f}%")
print(f"   Linhas mantidas: {len(data)} de {len(df_raw)}")


✅ Dataset limpo salvo como 'spacex_data_cleaned.csv'

📊 ESTATÍSTICAS:
   Redução de dados: 49.7%
   Linhas mantidas: 94 de 187


* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* **From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.**

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [87]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

These functions will apply the outputs globally to the above variables. Let's take a looks at <code>BoosterVersion</code> variable. Before we apply  <code>getBoosterVersion</code> the list is empty:


In [21]:
BoosterVersion

[]

Now, let's apply <code> getBoosterVersion</code> function method to get the booster version


In [88]:
# Call getBoosterVersion
def getBoosterVersion(dataset):
    for x in dataset['rocket']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/rockets/" + str(x)).json()
            BoosterVersion.append(response['name'])

the list has now been update 


In [89]:
BoosterVersion[0:5]

[]

we can apply the rest of the  functions here:


In [90]:
# Call getLaunchSite
def getLaunchSite(dataset):
    for x in dataset['launchpad']:
        if x:
            response = requests.get("https://api.spacexdata.com/v4/launchpads/" + str(x)).json()
            Longitude.append(response['longitude'])
            Latitude.append(response['latitude'])
            LaunchSite.append(response['name'])

In [92]:
# Call getPayloadData
def getCoreData(dataset):
    for core_dict in dataset['cores']:
        if core_dict and core_dict['core'] != None:
            response = requests.get("https://api.spacexdata.com/v4/cores/" + core_dict['core']).json()
            Block.append(response['block'])
            ReusedCount.append(response['reuse_count'])
            Serial.append(response['serial'])
        else:
            Block.append(None)
            ReusedCount.append(None)
            Serial.append(None)

# Executar as funções
print("\n1️⃣ Extraindo booster versions...")
getBoosterVersion(data)
print(f"   ✅ {len(BoosterVersion)} extraídos")

print("\n2️⃣ Extraindo launch sites...")
getLaunchSite(data)
print(f"   ✅ {len(LaunchSite)} extraídos")

print("\n3️⃣ Extraindo payload data...")
getPayloadData(data)
print(f"   ✅ {len(PayloadMass)} extraídos")

print("\n4️⃣ Extraindo core data...")
getCoreData(data)
print(f"   ✅ {len(Block)} extraídos")

print("\n✅ TODAS AS FEATURES EXTRAÍDAS!")


1️⃣ Extraindo booster versions...
   ✅ 94 extraídos

2️⃣ Extraindo launch sites...
   ✅ 94 extraídos

3️⃣ Extraindo payload data...
   ✅ 94 extraídos

4️⃣ Extraindo core data...
   ✅ 94 extraídos

✅ TODAS AS FEATURES EXTRAÍDAS!


In [26]:
# Call getCoreData
getCoreData(data)

Finally lets construct our dataset using the data we have obtained. We we combine the columns into a dictionary.


In [93]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [95]:
print("\n" + "="*70)
print("CRIANDO DATAFRAME FINAL")
print("="*70)

# Criar DataFrame com todas as features
X = pd.DataFrame({
    'FlightNumber': data['flight_number'].values,
    'Date': data['date'].values,
    'BoosterVersion': BoosterVersion,
    'LaunchSite': LaunchSite,
    'Longitude': Longitude,
    'Latitude': Latitude,
    'PayloadMass': PayloadMass,
    'Orbit': Orbit,
    'Block': Block,
    'ReusedCount': ReusedCount,
    'Serial': Serial
})

print(f"\n✅ DataFrame final criado!")
print(f"   Dimensões: {X.shape[0]} linhas x {X.shape[1]} colunas")

# Ver o resultado
print(f"\n📋 Primeiras 10 linhas:")
print(X.head(10))

print(f"\n📈 Informações:")
print(X.info())

print(f"\n📊 Primeiras estatísticas:")
print(X.describe())



CRIANDO DATAFRAME FINAL

✅ DataFrame final criado!
   Dimensões: 94 linhas x 11 colunas

📋 Primeiras 10 linhas:
   FlightNumber        Date BoosterVersion       LaunchSite   Longitude  \
0             1  2006-03-24       Falcon 1  Kwajalein Atoll  167.743129   
1             2  2007-03-21       Falcon 1  Kwajalein Atoll  167.743129   
2             4  2008-09-28       Falcon 1  Kwajalein Atoll  167.743129   
3             5  2009-07-13       Falcon 1  Kwajalein Atoll  167.743129   
4             6  2010-06-04       Falcon 9     CCSFS SLC 40  -80.577366   
5             8  2012-05-22       Falcon 9     CCSFS SLC 40  -80.577366   
6            10  2013-03-01       Falcon 9     CCSFS SLC 40  -80.577366   
7            11  2013-09-29       Falcon 9      VAFB SLC 4E -120.610829   
8            12  2013-12-03       Falcon 9     CCSFS SLC 40  -80.577366   
9            13  2014-01-06       Falcon 9     CCSFS SLC 40  -80.577366   

    Latitude  PayloadMass Orbit  Block  ReusedCount    Serial

Show the summary of the dataframe


In [96]:
print("\n" + "="*70)
print("EXTRAINDO TARGET (Landing Success)")
print("="*70)

# Criar lista para armazenar resultado de pouso
LandingClass = []

for core_dict in data['cores']:
    if core_dict['landing_success'] == True:
        LandingClass.append(1)  # Sucesso
    else:
        LandingClass.append(0)  # Falha

# Adicionar como coluna no X
X['LandingClass'] = LandingClass

print(f"\n✅ Target adicionado ao DataFrame!")
print(f"\n📊 Distribuição de pouso:")
print(f"   Sucessos (1): {sum(LandingClass)}")
print(f"   Falhas (0): {len(LandingClass) - sum(LandingClass)}")
print(f"   Taxa de sucesso: {(sum(LandingClass)/len(LandingClass)*100):.1f}%")

print(f"\n📋 Últimas 10 linhas com target:")
print(X.tail(10))



EXTRAINDO TARGET (Landing Success)

✅ Target adicionado ao DataFrame!

📊 Distribuição de pouso:
   Sucessos (1): 60
   Falhas (0): 34
   Taxa de sucesso: 63.8%

📋 Últimas 10 linhas com target:
    FlightNumber        Date BoosterVersion    LaunchSite  Longitude  \
84            96  2020-06-13       Falcon 9  CCSFS SLC 40 -80.577366   
85            97  2020-06-30       Falcon 9  CCSFS SLC 40 -80.577366   
86            98  2020-07-20       Falcon 9  CCSFS SLC 40 -80.577366   
87           100  2020-08-18       Falcon 9  CCSFS SLC 40 -80.577366   
88           101  2020-08-30       Falcon 9  CCSFS SLC 40 -80.577366   
89           102  2020-09-03       Falcon 9    KSC LC 39A -80.603956   
90           103  2020-10-06       Falcon 9    KSC LC 39A -80.603956   
91           104  2020-10-18       Falcon 9    KSC LC 39A -80.603956   
92           105  2020-10-24       Falcon 9  CCSFS SLC 40 -80.577366   
93           106  2020-11-05       Falcon 9  CCSFS SLC 40 -80.577366   

     Latitude

In [97]:
# Salvar o DataFrame
X.to_csv('spacex_data_final.csv', index=False)
print("\n✅ Dataset final salvo como 'spacex_data_final.csv'")

print(f"\n📊 RESUMO FINAL:")
print(f"   Linhas de dados: {len(X)}")
print(f"   Features: {len(X.columns)-1}")
print(f"   Target (LandingClass): Sim")
print(f"   Período: 2006 - 2020")
print(f"   Pronto para Machine Learning: ✅")



✅ Dataset final salvo como 'spacex_data_final.csv'

📊 RESUMO FINAL:
   Linhas de dados: 94
   Features: 11
   Target (LandingClass): Sim
   Período: 2006 - 2020
   Pronto para Machine Learning: ✅


In [98]:
# 1. Análise Exploratória (EDA)
print("\n1️⃣ ANÁLISE EXPLORATÓRIA:")
print(f"   Boosters únicos: {X['BoosterVersion'].nunique()}")
print(f"   Locais de lançamento: {X['LaunchSite'].nunique()}")
print(f"   Órbitas diferentes: {X['Orbit'].nunique()}")

# 2. Visualizações
print(f"\n2️⃣ VISUALIZAÇÕES:")
print("   Booster Version distribution")
print("   Launch Site distribution")
print("   Landing Success rate by LaunchSite")

# 3. Machine Learning
print(f"\n3️⃣ MACHINE LEARNING:")
print("   Train/Test split")
print("   Model training (Decision Tree, SVM, etc)")
print("   Model evaluation")



1️⃣ ANÁLISE EXPLORATÓRIA:
   Boosters únicos: 2
   Locais de lançamento: 4
   Órbitas diferentes: 11

2️⃣ VISUALIZAÇÕES:
   Booster Version distribution
   Launch Site distribution
   Landing Success rate by LaunchSite

3️⃣ MACHINE LEARNING:
   Train/Test split
   Model training (Decision Tree, SVM, etc)
   Model evaluation


### Task 2: Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [ ]:
# Hint data['BoosterVersion']!='Falcon 1'


Now that we have removed some values we should reset the FlgihtNumber column


In [100]:
print("="*70)
print("FILTRANDO APENAS FALCON 9")
print("="*70)

# PASSO 1: Ver distribuição de boosters
print("\n1️⃣ Distribuição de boosters antes da filtragem:")
print(X['BoosterVersion'].value_counts())

# PASSO 2: Filtrar apenas Falcon 9
print(f"\n2️⃣ Filtrando dados:")
print(f"   Antes: {len(X)} linhas")

# Usar a hint: X['BoosterVersion'] != 'Falcon 1'
data_falcon9 = X[X['BoosterVersion'] != 'Falcon 1'].copy()

print(f"   Depois: {len(data_falcon9)} linhas")
print(f"   Removidas: {len(X) - len(data_falcon9)} linhas (Falcon 1)")

# PASSO 3: Resetar o índice
print(f"\n3️⃣ Resetando índice:")
print(f"   Índice antigo: {data_falcon9.index.tolist()[:10]}...")

data_falcon9 = data_falcon9.reset_index(drop=True)

print(f"   Índice novo: {data_falcon9.index.tolist()[:10]}...")
print(f"   ✅ Índice resetado de 0 a {len(data_falcon9)-1}")

# PASSO 4: Resetar FlightNumber column (renumerar de 1)
print(f"\n4️⃣ Resetando FlightNumber column:")
print(f"   Números antigos: {data_falcon9['FlightNumber'].tolist()[:10]}")

# Renumerar de 1 a total de linhas
data_falcon9['FlightNumber'] = range(1, len(data_falcon9) + 1)

print(f"   Números novos: {data_falcon9['FlightNumber'].tolist()[:10]}")
print(f"   ✅ FlightNumber resetado de 1 a {len(data_falcon9)}")

# PASSO 5: Verificar resultado
print(f"\n5️⃣ Verificação final:")
print(f"   Boosters únicos: {data_falcon9['BoosterVersion'].unique()}")
print(f"   Total de linhas: {len(data_falcon9)}")
print(f"   Índice começa em: {data_falcon9.index[0]}")
print(f"   Índice termina em: {data_falcon9.index[-1]}")

# PASSO 6: Visualizar resultado
print(f"\n📋 Primeiras 10 linhas de data_falcon9:")
print(data_falcon9.head(10))

print(f"\n📋 Últimas 5 linhas:")
print(data_falcon9.tail())

# PASSO 7: Informações
print(f"\n📈 Informações do dataset final:")
print(data_falcon9.info())

print("\n" + "="*70)
print("✅ FALCON 9 FILTRADO E PRONTO!")
print("="*70)


FILTRANDO APENAS FALCON 9

1️⃣ Distribuição de boosters antes da filtragem:
Falcon 9    90
Falcon 1     4
Name: BoosterVersion, dtype: int64

2️⃣ Filtrando dados:
   Antes: 94 linhas
   Depois: 90 linhas
   Removidas: 4 linhas (Falcon 1)

3️⃣ Resetando índice:
   Índice antigo: [4, 5, 6, 7, 8, 9, 10, 11, 12, 13]...
   Índice novo: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]...
   ✅ Índice resetado de 0 a 89

4️⃣ Resetando FlightNumber column:
   Números antigos: [6, 8, 10, 11, 12, 13, 14, 15, 16, 17]
   Números novos: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
   ✅ FlightNumber resetado de 1 a 90

5️⃣ Verificação final:
   Boosters únicos: ['Falcon 9']
   Total de linhas: 90
   Índice começa em: 0
   Índice termina em: 89

📋 Primeiras 10 linhas de data_falcon9:
   FlightNumber        Date BoosterVersion    LaunchSite   Longitude  \
0             1  2010-06-04       Falcon 9  CCSFS SLC 40  -80.577366   
1             2  2012-05-22       Falcon 9  CCSFS SLC 40  -80.577366   
2             3  2013-03-01       Fa

In [101]:
# Salvar data_falcon9
data_falcon9.to_csv('data_falcon9.csv', index=False)
print("\n✅ DataFrame 'data_falcon9' salvo como 'data_falcon9.csv'")

# Resumo
print(f"\n📊 RESUMO:")
print(f"   Linhas: {len(data_falcon9)}")
print(f"   Colunas: {len(data_falcon9.columns)}")
print(f"   Apenas Falcon 9: ✅")
print(f"   Índice resetado: ✅")
print(f"   FlightNumber corrigido: ✅")
print(f"   Pronto para próxima task: ✅")



✅ DataFrame 'data_falcon9' salvo como 'data_falcon9.csv'

📊 RESUMO:
   Linhas: 90
   Colunas: 12
   Apenas Falcon 9: ✅
   Índice resetado: ✅
   FlightNumber corrigido: ✅
   Pronto para próxima task: ✅


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [103]:
print("="*70)
print("LIMPEZA DE DADOS - DATA WRANGLING")
print("="*70)

# Copiar dados
data_processed = data_falcon9.copy()

print("\n📊 Valores faltantes ANTES:")
print(data_processed.isnull().sum())

# Preencher valores numéricos com 0
data_processed['PayloadMass'] = data_processed['PayloadMass'].fillna(0)
data_processed['Block'] = data_processed['Block'].fillna(0)
data_processed['ReusedCount'] = data_processed['ReusedCount'].fillna(0)

print("\n✅ Valores faltantes DEPOIS:")
print(data_processed.isnull().sum())

# Verificar resultado
if data_processed.isnull().sum().sum() == 0:
    print("\n🎉 DATASET COMPLETAMENTE LIMPO!")
    print(f"   Total de linhas: {len(data_processed)}")
    print(f"   Total de colunas: {len(data_processed.columns)}")
else:
    print(f"\n⚠️ Ainda há {data_processed.isnull().sum().sum()} valores faltantes")

# Salvar
data_processed.to_csv('data_falcon9_processed.csv', index=False)
print("\n✅ Dataset salvo como 'data_falcon9_processed.csv'")


LIMPEZA DE DADOS - DATA WRANGLING

📊 Valores faltantes ANTES:
FlightNumber      0
Date              0
BoosterVersion    0
LaunchSite        0
Longitude         0
Latitude          0
PayloadMass       5
Orbit             0
Block             0
ReusedCount       0
Serial            0
LandingClass      0
dtype: int64

✅ Valores faltantes DEPOIS:
FlightNumber      0
Date              0
BoosterVersion    0
LaunchSite        0
Longitude         0
Latitude          0
PayloadMass       0
Orbit             0
Block             0
ReusedCount       0
Serial            0
LandingClass      0
dtype: int64

🎉 DATASET COMPLETAMENTE LIMPO!
   Total de linhas: 90
   Total de colunas: 12

✅ Dataset salvo como 'data_falcon9_processed.csv'


Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


In [104]:
# Calculate the mean value of PayloadMass column
import numpy as np

print("="*70)
print("TASK 3: DEALING WITH MISSING VALUES")
print("="*70)

# PASSO 1: Calcular a média de PayloadMass
print("\n1️⃣ Calculando a média de PayloadMass:")
mean_PayloadMass = data_processed['PayloadMass'].mean()
print(f"   Média: {mean_PayloadMass:.2f}")

# Ver quantos NaN tem
nan_count_before = data_processed['PayloadMass'].isnull().sum()
print(f"   Valores faltantes (NaN): {nan_count_before}")

# PASSO 2: Preencher NaN com a média usando .replace()
print("\n2️⃣ Preenchendo NaN com a média:")
data_processed['PayloadMass'] = data_processed['PayloadMass'].replace(np.nan, mean_PayloadMass)

# Ou alternativa: fillna()
# data_processed['PayloadMass'] = data_processed['PayloadMass'].fillna(mean_PayloadMass)

# PASSO 3: Verificar resultado
nan_count_after = data_processed['PayloadMass'].isnull().sum()
print(f"   Valores faltantes depois: {nan_count_after}")
print(f"   ✅ {nan_count_before} valores preenchidos!")

# PASSO 4: Visualizar os dados
print("\n3️⃣ Amostra dos dados:")
print(data_processed['PayloadMass'].head(20))

print("\n4️⃣ Estatísticas de PayloadMass:")
print(f"   Mínimo: {data_processed['PayloadMass'].min():.2f}")
print(f"   Máximo: {data_processed['PayloadMass'].max():.2f}")
print(f"   Média: {data_processed['PayloadMass'].mean():.2f}")
print(f"   Mediana: {data_processed['PayloadMass'].median():.2f}")
print(f"   Desvio padrão: {data_processed['PayloadMass'].std():.2f}")

print("\n5️⃣ Verificação final:")
print(f"   Total de NaN em todo dataset: {data_processed.isnull().sum().sum()}")

if data_processed.isnull().sum().sum() == 0:
    print("\n✅ DATASET COMPLETAMENTE LIMPO!")
else:
    print(f"\n⚠️ Ainda há valores faltantes em outras colunas")

print("\n" + "="*70)
print("✅ TASK 3 CONCLUÍDA!")
print("="*70)

# PASSO 6: Salvar dados atualizados
data_processed.to_csv('data_falcon9_processed.csv', index=False)
print("\n✅ Dataset atualizado salvo como 'data_falcon9_processed.csv'")


TASK 3: DEALING WITH MISSING VALUES

1️⃣ Calculando a média de PayloadMass:
   Média: 5783.35
   Valores faltantes (NaN): 0

2️⃣ Preenchendo NaN com a média:
   Valores faltantes depois: 0
   ✅ 0 valores preenchidos!

3️⃣ Amostra dos dados:
0        0.0
1      525.0
2      677.0
3      500.0
4     3170.0
5     3325.0
6     2296.0
7     1316.0
8     4535.0
9     4428.0
10    2216.0
11    2395.0
12     570.0
13    1898.0
14    4707.0
15    2477.0
16    2034.0
17     553.0
18    5271.0
19    3136.0
Name: PayloadMass, dtype: float64

4️⃣ Estatísticas de PayloadMass:
   Mínimo: 0.00
   Máximo: 15600.00
   Média: 5783.35
   Mediana: 4315.00
   Desvio padrão: 4937.86

5️⃣ Verificação final:
   Total de NaN em todo dataset: 0

✅ DATASET COMPLETAMENTE LIMPO!

✅ TASK 3 CONCLUÍDA!

✅ Dataset atualizado salvo como 'data_falcon9_processed.csv'


In [105]:
data_falcon9['PayloadMass'].isnull().sum()


5

In [106]:
data_falcon9.to_csv('dataset_part_1.csv', index=False)

You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range. 


<code>data_falcon9.to_csv('dataset_part_1.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD. 


<!--## Change Log
-->


<!--

|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2020-09-20|1.1|Joseph|get result each time you run|
|2020-09-20|1.1|Azim |Created Part 1 Lab using SpaceX API|
|2020-09-20|1.0|Joseph |Modified Multiple Areas|
-->


Copyright ©IBM Corporation. All rights reserved.
